In [1]:
from langgraph.prebuilt import create_react_agent

from langchain_aws.memory import (
    create_store_messages_tool,
    create_search_memory_tool,
    create_list_messages_tool
)
from bedrock_agentcore.memory import MemoryClient

In [2]:
REGION = "us-east-1"

In [3]:
memory_client = MemoryClient(region_name=REGION)

In [ ]:
create_response = memory_client.create_memory_and_wait(
    name="CustomerSupportAgentMemory",
    strategies=[{
        "summaryMemoryStrategy": {
            "name": "SessionSummarizer",
            "namespaces": ["/summaries/{actorId}/{sessionId}"]
        }
    }]
)

In [4]:
MEMORY_ID = create_response["id"]
ACTOR_ID = "User4"
SESSION_ID = "OrderSupportSession4"

In [5]:
# Create memory management tool
store_messages_tool = create_store_messages_tool(
    memory_client=memory_client
)

# Create memory search tool
search_memory_tool = create_search_memory_tool(
    memory_client=memory_client
)

# Create the list messages tool
list_messages_tool = create_list_messages_tool(
    memory_client=memory_client
)

In [6]:
from langchain_core.tools import tool

@tool
def lookup_order(order_id: str) -> str:
    """Returns the order status"""
    return f"Customer order {order_id} was shipped 3 days ago, with an expected delivery tomorrow before 5pm"

@tool
def update_customer_email(email: str):
    """Updates customer's email address"""
    return f"Customer's email updated to {email}"

In [7]:
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage, SystemMessage

llm_model = "bedrock_converse:us.anthropic.claude-3-5-haiku-20241022-v1:0"
prompt_template = "You are a customer service assistant. Here is the conversation context: {summary}"

def pre_model_hook(state):
    messages = state["messages"]

    # Get summary or other long term memories
    last_message = messages[-1]
    summary = ""
    if isinstance(last_message, HumanMessage):
        memories = search_memory_tool.invoke({
            "query": last_message.content
        })
        if len(memories) > 0:
            summary = memories[0]

    # Get previous conversations, trimming to last 2
    previous_messages = list_messages_tool.invoke({"max_results": 2}, config=config)
    previous_messages.reverse()

    # System message, that will have the long
    # term memory context
    system_message = SystemMessage(
        content=prompt_template.format(summary=summary)
    )

    messages = [system_message] + previous_messages + messages

    return {
        "llm_input_messages": messages
    }

def post_model_hook(state):
    store_messages_tool.invoke({
        "messages": state["messages"]
    })

agent = create_react_agent(
    llm_model,
    tools=[lookup_order, update_customer_email],
    pre_model_hook=pre_model_hook,
    post_model_hook=post_model_hook
)

In [8]:
config = {
    "configurable": {
        "memory_id": MEMORY_ID,
        "session_id": SESSION_ID,
        "actor_id": ACTOR_ID,
        "namespace": f"/summaries/{ACTOR_ID}/{SESSION_ID}"
    }
}

In [9]:
def list_events():
    events = memory_client.list_events(memory_id=MEMORY_ID, session_id=SESSION_ID, actor_id=ACTOR_ID)
    for event in events:
        print(f"\n================================ Event Id: {event['eventId']} =================================\n")
        for payload in event['payload']:
            print(f"================================ {payload['conversational']['role']} =================================")
            print(payload['conversational']['content']['text'])

In [10]:
response = agent.invoke({
    "messages": [
        "Hi, I'm having trouble with my order #12345"
    ]
}, config)

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi, I'm having trouble with my order #12345
================================== Ai Message ==================================

[{'type': 'text', 'text': "Let me help you check the status of your order. I'll look up the details for order #12345 right away."}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_beyZ90h7TPGdwIMjqYK6Xw'}]
Tool Calls:
  lookup_order (tooluse_beyZ90h7TPGdwIMjqYK6Xw)
 Call ID: tooluse_beyZ90h7TPGdwIMjqYK6Xw
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

I see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about 

In [11]:
list_events()


================================ Event Id: 0000001753926824000#46374f3d =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345
================================ TOOL =================================
Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================ ASSISTANT =================================
I see that your order #12345 was shipped 3 days ago and is expected to be delivered tomorrow before 5 PM. Is there anything specific about the order that's causing you concern? I'd be happy to help you further or provide more information about the shipment.

================================ Event Id: 0000001753926822000#47403cb4 =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345


In [12]:
response = agent.invoke({
    "messages": [
        "Actually, before that - I also want to change my email address"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Actually, before that - I also want to change my email address
================================== Ai Message ==================================

I'll help you with updating your email address first, and then we'll check the status of your order.

Could you please provide me with the new email address you'd like to use?


In [13]:
list_events()


================================ Event Id: 0000001753926839000#d5052028 =================================

================================ USER =================================
Actually, before that - I also want to change my email address
================================ ASSISTANT =================================
I'll help you with updating your email address first, and then we'll check the status of your order.

Could you please provide me with the new email address you'd like to use?

================================ Event Id: 0000001753926824000#46374f3d =================================

================================ USER =================================
Hi, I'm having trouble with my order #12345
================================ TOOL =================================
Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================ ASSISTANT =================================
I see that your order #12345 was shipped

In [14]:
response = agent.invoke({
    "messages": [
        "myemail@example.com"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

myemail@example.com
================================== Ai Message ==================================

[{'type': 'text', 'text': "I'll help you update your email address right away."}, {'type': 'tool_use', 'name': 'update_customer_email', 'input': {'email': 'myemail@example.com'}, 'id': 'tooluse_UEBGmgAQTR28Mobm7THbgw'}]
Tool Calls:
  update_customer_email (tooluse_UEBGmgAQTR28Mobm7THbgw)
 Call ID: tooluse_UEBGmgAQTR28Mobm7THbgw
  Args:
    email: myemail@example.com
================================= Tool Message =================================
Name: update_customer_email

Customer's email updated to myemail@example.com
================================== Ai Message ==================================

Your email address has been successfully updated to myemail@example.com. 

Is there anything else I can help you with today? Would you like to check the status of an order?


In [15]:
list_events()


================================ Event Id: 0000001753926858000#eed1ac65 =================================

================================ USER =================================
myemail@example.com
================================ TOOL =================================
Customer's email updated to myemail@example.com
================================ ASSISTANT =================================
Your email address has been successfully updated to myemail@example.com. 

Is there anything else I can help you with today? Would you like to check the status of an order?

================================ Event Id: 0000001753926857000#e22d00a2 =================================

================================ USER =================================
myemail@example.com

================================ Event Id: 0000001753926839000#d5052028 =================================

================================ USER =================================
Actually, before that - I also want to change my e

In [17]:
response = agent.invoke({
    "messages": [
        "The package already arrived, but seems damaged"
    ]
}, config)
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

The package already arrived, but seems damaged
================================== Ai Message ==================================

[{'type': 'text', 'text': 'I apologize to hear that the package arrived damaged. Let me help you verify the order details first.'}, {'type': 'tool_use', 'name': 'lookup_order', 'input': {'order_id': '12345'}, 'id': 'tooluse_ryehEI-SRd2ckquOtKs0MA'}]
Tool Calls:
  lookup_order (tooluse_ryehEI-SRd2ckquOtKs0MA)
 Call ID: tooluse_ryehEI-SRd2ckquOtKs0MA
  Args:
    order_id: 12345
================================= Tool Message =================================
Name: lookup_order

Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================== Ai Message ==================================

I see that the order was shipped recently. However, since you mentioned the package has already arrived and is damaged, I recommend the 

In [18]:
list_events()


================================ Event Id: 0000001753926912000#095ae727 =================================

================================ USER =================================
The package already arrived, but seems damaged
================================ TOOL =================================
Customer order 12345 was shipped 3 days ago, with an expected delivery tomorrow before 5pm
================================ ASSISTANT =================================
I see that the order was shipped recently. However, since you mentioned the package has already arrived and is damaged, I recommend the following steps:

1. Take clear photos of the damaged package and the damaged items.
2. Contact our customer support team with the order details and photos.
3. We can help you with a replacement or refund.

Would you like me to help you further with this issue? Could you provide me with the specific order number so I can assist you more accurately?

================================ Event Id: 00